### Pydantic Basics: Creating and using Models

> Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.👌

In [1]:
from pydantic import BaseModel

In [ ]:
# Alternate approach - using dataclasses and decorator

from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

person=Person(name="Karan",age=34,city="Gurugram")
print(person)

Person(name='Karan', age=34, city='Gurugram')


In [10]:
person=Person(name="Karan",age=34,city=12)
print(person)

Person(name='Karan', age=34, city=12)


<ins>**No error with dataclass**</ins>. Data validation is not at all happening.

In [11]:
class Person1(BaseModel):
    name:str
    age:int
    city:str

person_py=Person1(name="Karan",age=34,city="Gurugram")
print(person_py)

name='Karan' age=34 city='Gurugram'


In [12]:
type(person_py)

__main__.Person1

In [17]:
person_oth=Person1(name="Karan",age=34,city=12)
print(person_oth)
# Data validation is happening

ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

### 2. Model with optional fields
> Add optional fields using Python's Optional type:

In [18]:
from typing import Optional
class Employee(BaseModel):
    id: int
    name: str
    department: str
    salary: Optional[float] = None # Optional with default value
    is_active: Optional[bool] = True # Optional with default True

In [20]:
# Examples with and without optional fields
emp1 = Employee(id=1, name="Drake", department="IT")
print(emp1)

id=1 name='Drake' department='IT' salary=None is_active=True


In [21]:
emp2 = Employee(id=1, name="Sanya", department="IT", salary=50000, is_active=False)
print(emp2)

id=1 name='Sanya' department='IT' salary=50000.0 is_active=False


### Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

Pydantic validates types even for optional fields when values are provided

In [22]:
from pydantic import BaseModel
from typing import List

class Classroom(BaseModel):
    room_number: str
    students: List[str]  # List of strings
    capacity: int

In [23]:
# Create a classroom
classroom = Classroom(
    room_number="A101",
    students=["Jonathan", "Littlehand", "Steve"],
    capacity=30
)
print(classroom)

room_number='A101' students=['Jonathan', 'Littlehand', 'Steve'] capacity=30


In [25]:
# Create a classroom
classroom_tu = Classroom(
    room_number="A101",
    students=("Jonathan", "Littlehand", "Mary"),
    capacity=30
)
print(classroom_tu) # type casting is also happening

room_number='A101' students=['Jonathan', 'Littlehand', 'Mary'] capacity=30


In [26]:
try:
    invalid_val=Classroom(room_number="A1",students=["Jamie",123],capacity=30)
except ValueError as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### 3. Model with Nested Models

> Create complex structures with nested models:

In [27]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    pin_code: str

class Customer(BaseModel):
    customer_id: int
    name: str
    address: Address  # Nested model

# Create a customer with nested address
customer = Customer(
    customer_id=1,
    name="Jules",
    address={"street":"123 Main St","city":"Boston","pin_code":"02108"}
)
print(customer)

customer_id=1 name='Jules' address=Address(street='123 Main St', city='Boston', pin_code='02108')


In [28]:
from pydantic import BaseModel

class Address1(BaseModel):
    street: str
    city: str
    pin_code: int

class Customer1(BaseModel):
    customer_id: int
    name: str
    address: Address1  # Nested model

# Create a customer with nested address
customer = Customer1(
    customer_id=1,
    name="Jules",
    address={"street":"123 Main St","city":"Boston","pin_code":"02108"}
)
print(customer)

customer_id=1 name='Jules' address=Address1(street='123 Main St', city='Boston', pin_code=2108)


### Pydantic Fields: Customization and Constraints

> The field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.🌻

In [ ]:
from pydantic import BaseModel, Field
class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=1000) #gt (greather) than 0; le (less) than or equal to 1000
    quantity:int=Field(ge=0)

# Valid instance
item = Item(name="Pen",price=28.75,quantity=25)

print(item)

name='Pen' price=28.75 quantity=25


In [30]:
from pydantic import BaseModel, Field
class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=1000) #gt (greather) than 0; le (less) than or equal to 1000
    quantity:int=Field(ge=0)

# Valid instance
item = Item(name="Pen",price=-1,quantity=25)

print(item)

ValidationError: 1 validation error for Item
price
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than

In [31]:
# Let's use fields with some default value

from pydantic import BaseModel, Field

class User(BaseModel):
    username: str = Field(...,description="Unique username for the user")
    age: int = Field(default=18,description="User age, defaults to 18")
    email: str = Field(default_factory=lambda: "user@example.com", description="Default email address")

# Examples
user1 = User(username="samay")
print(user1)

user2 = User(username="jack", age=64, email="jack@domain.au")
print(user2)

username='samay' age=18 email='user@example.com'
username='jack' age=64 email='jack@domain.au'


In [33]:
print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique username for the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
